# Breit–Wigner interference

This notebook is inspired by a talk by Robert about interference between amplitudes. As an illustrative example, we investigate how the intensity is influenced by a constant background term $C$:

$$
\begin{array}{rcl}
  I(m) &=& \left|\text{BW}(m) + C\right|^2 \\
  \text{BW}(m) &=& \frac{m_0\Gamma_0}{m^2-m_0^2-im_0\Gamma_0} \\
  C &=& |C|e^{i\phi}
\end{array}
$$

In [ ]:
%pip install -q ipympl

In [ ]:
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget

In [ ]:
sliders = {
    "m0": w.FloatSlider(4.3, min=0.0, max=10.0, step=0.1, description="$m_0$"),
    "Γ0": w.FloatSlider(0.9, min=0.1, max=5.0, step=0.1, description=R"$\Gamma_0$"),
    "c_mag": w.FloatSlider(0.6, min=0, max=2, step=0.1, description="$|c|$"),
    "phi": w.FloatSlider(10, min=-180, max=180, step=5, description=R"$\phi$"),
}
ui = w.VBox([
    sliders["m0"],
    sliders["Γ0"],
    w.HBox([sliders["c_mag"], sliders["phi"]]),
])

In [ ]:
def update_plot(m0, Γ0, c_mag, phi):
    global lines
    c: float = c_mag * np.exp(1j * np.deg2rad(phi))
    argand = breit_wigner(m_argand, m0, Γ0) + c
    bw = breit_wigner(m, m0, Γ0)
    signal = np.abs(bw) ** 2
    total = np.abs(bw + c) ** 2
    if lines is None:
        lines = (
            ax1.plot(argand.real, argand.imag, c="C0", lw=2)[0],
            ax1.plot([0, c.real], [0, c.imag], c="C1", lw=1, ls="dotted")[0],
            ax1.scatter([c.real], [c.imag], c="C1", lw=3),
            ax2.plot(m, total, c="black", lw=2, label=R"$|\text{BW} + C|^2$")[0],
            ax2.plot(m, signal, c="C0", lw=2, ls="--", label=R"$|\text{BW}|^2$")[0],
            ax2.axhline(c_mag**2, c="C1", lw=1, ls="--", label=R"$|C|$"),
        )
    else:
        lines[0].set_data(argand.real, argand.imag)
        lines[1].set_data([0, c.real], [0, c.imag])
        lines[2].set_offsets([[c.real, c.imag]])
        lines[3].set_data(m, total)
        lines[4].set_data(m, signal)
        lines[5].set_ydata([c_mag**2])
    ax1.relim()
    ax1.autoscale_view()
    ax2.set_ylim(-0.3, max(1.8, 1.05 * max(signal.max(), total.max())))
    fig.canvas.draw_idle()


def breit_wigner(m, m0, Γ0):
    return m0 * Γ0 / (m**2 - m0**2 + 1j * m0 * Γ0)


m = np.linspace(0, 10, num=500)
m_argand = np.linspace(0, 10, num=5_000)
lines = None

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax1, ax2 = axes
for ax in axes:
    ax.spines["left"].set_position("zero")
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

ax1.set_title("Argand plot")
ax1.set_xlabel("Real part")
ax1.set_ylabel("Imaginary part")
ax1.set_aspect("equal", adjustable="datalim")
ax1.axhline(0, alpha=0)  # for scaling
ax1.axvline(0, alpha=0)  # for scaling

ax2.set_title("Intensity")
ax2.set_xlabel("$m$")
ax2.set_ylabel(R"Intensity $|\text{BW} + C|^2$")

output = w.interactive_output(update_plot, sliders)
ax2.legend()
fig.tight_layout()
display(ui, output)